<a href="https://colab.research.google.com/github/claudelepere/ML_GitHub/blob/main/NEW_Prediction_thresholds_tuning_trainer_EN_11_24000__24100_ml1024_ep5_lr1e_5_th05_at1024_fp16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# If these errors occur, run the next code cell.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.

torch 2.5.1+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.

torch 2.5.1+cu124 requires nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-nvrtc-cu12 12.5.82 which is incompatible.

torch 2.5.1+cu124 requires nvidia-cuda-runtime-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-runtime-cu12 12.5.82 which is incompatible.

torch 2.5.1+cu124 requires nvidia-cudnn-cu12==9.1.0.70; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cudnn-cu12 9.3.0.75 which is incompatible.

torch 2.5.1+cu124 requires nvidia-cufft-cu12==11.2.1.3; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cufft-cu12 11.2.3.61 which is incompatible.

torch 2.5.1+cu124 requires nvidia-curand-cu12==10.3.5.147; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-curand-cu12 10.3.6.82 which is incompatible.

torch 2.5.1+cu124 requires nvidia-cusolver-cu12==11.6.1.9; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cusolver-cu12 11.6.3.83 which is incompatible.

torch 2.5.1+cu124 requires nvidia-cusparse-cu12==12.3.1.170; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cusparse-cu12 12.5.1.3 which is incompatible.

torch 2.5.1+cu124 requires nvidia-nvjitlink-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-nvjitlink-cu12 12.5.82 which is incompatible.

In [1]:
#!pip uninstall -y nvidia-cublas-cu12

In [2]:
#!pip install -r requirements.txt

# Error: gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
Ignore this error as gcsfs is not used.

In [3]:
!pip install -q transformers datasets

import datetime
import numpy as np
import os
import torch

from datasets         import DatasetDict
from google.colab     import files, userdata
from huggingface_hub  import create_repo, login, whoami
from sklearn.metrics  import classification_report
from transformers     import LongformerTokenizerFast, LongformerForSequenceClassification
from torch.utils.data import DataLoader

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.5 MB/s eta 0:00:00


## Hugging Face Hub (HF Hub) authenticate

In [4]:
os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN")    # Store the key in os.environ
hf_token               = os.environ.get('HF_TOKEN')

login(token=hf_token)

# Check
user = whoami(token=hf_token)
assert user['name'] == 'claudelepere', f"{user['name']} is not claudelepere"

print(f"user: {user}")

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


user: {'type': 'user', 'id': '66ec3d5f61228b02f8780beb', 'name': 'claudelepere', 'fullname': 'Claude Lepère', 'isPro': False, 'avatarUrl': 'https://cdn-avatars.huggingface.co/v1/production/uploads/66ec3d5f61228b02f8780beb/gvnf9pvm2KvE90ETMUQo3.jpeg', 'orgs': [], 'auth': {'type': 'access_token', 'accessToken': {'displayName': 'jobs_token', 'role': 'fineGrained', 'createdAt': '2025-01-04T17:44:35.493Z', 'fineGrained': {'canReadGatedRepos': False, 'global': [], 'scoped': [{'entity': {'_id': '66ec3d5f61228b02f8780beb', 'type': 'user', 'name': 'claudelepere'}, 'permissions': ['repo.content.read', 'repo.write']}]}}}}


## Create the model and dataset repos on HF Hub

In [5]:
repo_id = 'claudelepere/jobs_EN_11_0_24000_tuned_thresholds' # repo_id = my namespace/username_or_org/reponame
#model_repoUrl   = create_repo(repo_id=repo_id, repo_type="model",   private=True, exist_ok=True)
#dataset_repoUrl = create_repo(repo_id=repo_id, repo_type="dataset", private=True, exist_ok=True)

#print(f"Model Repo Url: {model_repoUrl} created successfully as a private repo")
#print(f"Dataset Repo Url: {dataset_repoUrl} created successfully as a private repo")

## Checks

In [6]:
!python -V

print(f"currentdir: {os.getcwd()}")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"device: {device}")

Python 3.11.11
currentdir: /content
device: cuda


In [7]:
skills         = 11
all_rows_low   = 24000
all_rows_high  = 24100
num_datapoints = all_rows_high - all_rows_low
max_length     = 1024
batch_size     = 4*8

datasetDict_zip_file_name = f"dataset_EN_{skills}_{all_rows_low}_{all_rows_high}.zip"
datasetDict_dir_name      = os.path.splitext(datasetDict_zip_file_name)[0]

print(f"datasetDict_zip_file_name: {datasetDict_zip_file_name}")
print(f"datasetDict_dir_name     : {datasetDict_dir_name}")
print()

#revision_name = "EN_11_0_24000_ml1024_ep5_lr1e-05_th0.5_at1024_fp16_ba8x4"
#print(f"revision_name: {revision_name}")

datasetDict_zip_file_name: dataset_EN_11_24000_24100.zip
datasetDict_dir_name     : dataset_EN_11_24000_24100



# Out Of Memory (OOM)

### OOM: check for and kill zombie processes

In [8]:
!ps aux | grep python
!kill -9 <PID>
if torch.cuda.is_available():
  !nvidia-smi
  print(torch.cuda.memory_summary())

root          88  1.0  0.0      0     0 ?        Z    17:53   0:13 [python3] <defunct>
root          89  0.0  0.0  63772 51096 ?        S    17:53   0:00 python3 /usr/local/bin/colab-file
root         134  0.2  0.1 779672 126952 ?       Sl   17:53   0:03 /usr/bin/python3 /usr/local/bin/j
root        5179 58.2  1.4 12139700 1243096 ?    Ssl  18:14   0:16 /usr/bin/python3 -m colab_kernel_
root        5217  1.8  0.0 545184 20424 ?        Sl   18:14   0:00 /usr/bin/python3 /usr/local/lib/p
root        5442  0.0  0.0   7376  3544 ?        S    18:14   0:00 /bin/bash -c ps aux | grep python
root        5444  0.0  0.0   6484  2356 ?        S    18:14   0:00 grep python
/bin/bash: -c: line 1: syntax error near unexpected token `newline'
/bin/bash: -c: line 1: `kill -9 <PID>'
Sun Mar  2 18:14:52 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----

### OOM: free up GPU memory

In [9]:
torch.cuda.empty_cache()

## Upload and unzip job dataset

In [10]:
def upload_unzip_dataset(file_name=datasetDict_zip_file_name):
  # Check if the file exists
  if not os.path.exists(file_name):
    print(f"'{file_name}' not found in /content. Uploading...")

    uploaded_files = files.upload()                              # Prompt file upload dialog
    if file_name not in uploaded_files:
      raise FileNotFoundError(f"'{file_name}' not found.")
    print(f"'{file_name}' successfully uploaded to /content")

    uploaded_file_name = list(uploaded_files.keys())[0]          # Get the name of the uploaded file

    !unzip {uploaded_file_name}

    unzipped_dir_name = os.path.splitext(uploaded_file_name)[0]
    assert unzipped_dir_name==datasetDict_dir_name, "unzipped_dir_name != datasetDict_dir_name"
  else:
    print(f"'{file_name}' already exists in /content.")

upload_unzip_dataset(datasetDict_zip_file_name)

'dataset_EN_11_24000_24100.zip' not found in /content. Uploading...


Saving dataset_EN_11_24000_24100.zip to dataset_EN_11_24000_24100.zip
'dataset_EN_11_24000_24100.zip' successfully uploaded to /content
Archive:  dataset_EN_11_24000_24100.zip
  inflating: dataset_EN_11_24000_24100/dataset_dict.json  
  inflating: dataset_EN_11_24000_24100/inference/data-00000-of-00001.arrow  
  inflating: dataset_EN_11_24000_24100/inference/dataset_info.json  
  inflating: dataset_EN_11_24000_24100/inference/state.json  


# Create datasetDict (HF DatasetDict) = 3 HF Dataset, train, validation and test

In [11]:
datasetDict = DatasetDict.load_from_disk(datasetDict_dir_name)
dataset     = datasetDict['inference']
print(f"dataset: {type(dataset)} shape={dataset.shape}\n{dataset}")

dataset: <class 'datasets.arrow_dataset.Dataset'> shape=(100, 8)
Dataset({
    features: ['id', 'text', '390', '135', '136', '137', '138', '139'],
    num_rows: 100
})


# Create labels (list), id2label (dict) and label2id (dict).
* dataset 7_1000_125_125  ,  48 labels
* dataset 7_128_18_54     ,  42 labels
* dataset 8910_1087_68_204, 206 labels
* dataset 11_1000         ,   6 labels

In [12]:
labels = [label for label in dataset.features.keys() if label not in ['id', 'text']]
labels.sort()
print(f"labels: {type(labels)} {len(labels)}\n{labels}")
num_labels = len(labels)

labels: <class 'list'> 6
['135', '136', '137', '138', '139', '390']


In [13]:
id2label = {idx: label for idx, label in enumerate(labels)}
print(f"id2label: {type(id2label)} {len(id2label)}\n{id2label}")

id2label: <class 'dict'> 6
{0: '135', 1: '136', 2: '137', 3: '138', 4: '139', 5: '390'}


In [14]:
label2id = {label: idx for idx, label in enumerate(labels)}
print(f"label2id: {type(label2id)} {len(label2id)}\n{label2id}")

label2id: <class 'dict'> 6
{'135': 0, '136': 1, '137': 2, '138': 3, '139': 4, '390': 5}


# A unique commit_message for all downloads to HF Hub

In [15]:
commit_message = "2025-02-08 20:47:44"

print(f"commit_message: {commit_message}")

commit_message: 2025-02-08 20:47:44


# Download the tokenizer and the model

In [16]:
print("Tokenizer")
tokenizer = LongformerTokenizerFast.from_pretrained(repo_id) #, revision=revision_name)

print("Model")
model = LongformerForSequenceClassification.from_pretrained(repo_id) #, revision=revision_name)
model.eval()  # Ensures no gradient computation

"""
inputs = tokenizer("Hello, my dog is cute", return_tensors="pt", truncation=True, padding=True)
outputs = model(**inputs)

print()
print(f"outputs: {type(outputs)} {outputs.keys()}\n{outputs}")
"""

Tokenizer


tokenizer_config.json:   0%|          | 0.00/1.28k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.56M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

Model


config.json:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/595M [00:00<?, ?B/s]

'\ninputs = tokenizer("Hello, my dog is cute", return_tensors="pt", truncation=True, padding=True)\noutputs = model(**inputs)\n\nprint()\nprint(f"outputs: {type(outputs)} {outputs.keys()}\n{outputs}")\n'

In [17]:
#import torch
#import numpy as np
#from transformers import LongformerForSequenceClassification

# Load model and set it to evaluation mode
#model = LongformerForSequenceClassification.from_pretrained(repo_id, revision=revision_name)
#

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Step 1: Tokenize dataset
def tokenize_function(examples):
  return tokenizer(examples["text"], padding='max_length', max_length=max_length, truncation=True)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Step 2: Convert to PyTorch DataLoader
tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"] + labels)
dataloader = DataLoader(tokenized_dataset, batch_size=batch_size)

# Step 3: Run Prediction
all_preds = []

with torch.no_grad():  # No gradients needed for prediction
    for batch in dataloader:
        # Move batch to GPU if available
        inputs = {k: batch[k].to(device) for k in ["input_ids", "attention_mask"]}

        # Forward pass
        outputs = model(**inputs)
        logits = outputs.logits  # Model outputs raw logits

        # Apply sigmoid to convert logits into probabilities
        probs = torch.sigmoid(logits)

        # Move results to CPU and convert to NumPy
        all_preds.extend(probs.cpu().numpy())

# Step 4: Convert Probabilities to Binary Predictions
##########################################################
#threshold = 0.5
#binary_preds = (np.array(all_preds) > threshold).astype(int)

###########################################################

custom_thresholds = np.array([0.4, 0.3, 0.3, 0.3, 0.4, 0.5])

print(f"Predictions Shape: {np.array(all_preds).shape}")  # Expected: (100, 6)
print(f"Thresholds Shape: {custom_thresholds.shape}")  # Expected: (6,)

binary_preds = (np.array(all_preds) > custom_thresholds).astype(int)

###########################################################
# Step 5: Compare with True Labels
true_labels = tokenized_dataset.to_pandas()[labels].values  # Converts to NumPy array
print(f"true_labels: {type(true_labels)} shape={true_labels.shape}")  # Should match binary_preds.shape

print(f"classification_report:\n{classification_report(true_labels, binary_preds, zero_division=0)}")


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Initializing global attention on CLS token...


Predictions Shape: (100, 6)
Thresholds Shape: (6,)
true_labels: <class 'numpy.ndarray'> shape=(100, 6)
classification_report:
              precision    recall  f1-score   support

           0       0.06      1.00      0.11         6
           1       0.22      1.00      0.36        22
           2       0.70      1.00      0.82        70
           3       0.90      1.00      0.95        90
           4       0.84      1.00      0.91        84
           5       0.00      0.00      0.00        12

   micro avg       0.54      0.96      0.69       284
   macro avg       0.45      0.83      0.53       284
weighted avg       0.72      0.96      0.80       284
 samples avg       0.54      0.97      0.69       284



In [18]:
raise Exception("I stop here")

Exception: I stop here

batch_size = 8

(100, 6)

              precision    recall  f1-score   support

           0       0.80      0.67      0.73         6
           1       0.92      0.55      0.69        22
           2       0.88      0.83      0.85        70
           3       0.94      0.99      0.96        90
           4       0.90      0.94      0.92        84
           5       0.67      0.33      0.44        12

   micro avg       0.90      0.87      0.88       284

   macro avg       0.85      0.72      0.77       284

weighted avg       0.90      0.87      0.87       284

 samples avg       0.91      0.89      0.89       284

batch_size = 4 x 8

(100, 6)
              precision    recall  f1-score   support

           0       0.80      0.67      0.73         6
           1       0.92      0.55      0.69        22
           2       0.88      0.83      0.85        70
           3       0.94      0.99      0.96        90
           4       0.90      0.94      0.92        84
           5       0.67      0.33      0.44        12

   micro avg       0.90      0.87      0.88       284

   macro avg       0.85      0.72      0.77       284

weighted avg       0.90      0.87      0.87       284

 samples avg       0.91      0.89      0.89       284


 With tuned thresholds, batch_size = 4 x 8

 Predictions Shape: (100, 6)
Thresholds Shape: (6,)
(100, 6)
              precision    recall  f1-score   support

           0       0.80      0.67      0.73         6
           1       0.70      0.64      0.67        22
           2       0.85      0.94      0.89        70
           3       0.93      0.99      0.96        90
           4       0.90      0.98      0.94        84
           5       0.67      0.33      0.44        12

   micro avg       0.88      0.91      0.89       284

   macro avg       0.81      0.76      0.77       284

weighted avg       0.87      0.91      0.89       284

 samples avg       0.89      0.93      0.90       284

# Making predictions

In [ ]:
dataLoader = DataLoader(tokenized_dataset, batch_size=batch_size, collate_fn=lambda x: {k: torch.cat([d[k] for d in x], dim=0) for k in x[0]})

all_preds = []

for batch in dataLoader:
  # Move tensors to the device if available
  batch = {k: v.to(device) if torch.is_tensor(v) else v for k, v in batch.items() if k in ["input_ids", "attention_mask"]}
  # Filter out unwanted keys like 'id'

  with torch.no_grad():
    outputs = model(**batch) # Pass the filtered batch to the model
    logits = outputs.logits
    preds = torch.sigmoid(logits).cpu().numpy()
    all_preds.append(preds)

threshold    = 0.5
binary_preds = (np.array(all_preds) > threshold).astype(int)
true_labels  = np.array(dataset['labels'])

# Compute classification report
#from sklearn.metrics import classification_report
print(classification_report(true_labels, binary_preds, zero_division=0))

raise Exception()

In [ ]:
print("It's the end")

In [ ]:
raise Exception()

In [ ]:
print(datasetDict['inference'])
print(datasetDict['inference']['labels'])

In [ ]:
with torch.no_grad():  # Disable gradient calculation during inference
    outputs = model(
        input_ids=inputs.input_ids.to(device),
        attention_mask=inputs.attention_mask.to(device)
    )


In [ ]:
probs = torch.sigmoid(outputs.logits)


In [ ]:
best_thresholds = [0.4, 0.3, 0.3, 0.3, 0.4, 0.5]
preds = np.zeros_like(probs)  # Initialize predictions array
for label_idx in range(num_labels):
  preds[:, label_idx] = (probs[:, label_idx] > best_thresholds[label_idx])  #.astype(int)

In [ ]:
print(f"probs: {type(probs)} shape={probs.shape}\n{probs}")
print(f"preds: {type(preds)} shape={preds.shape}\n{preds}")

In [ ]:
print(f"labels: {type(labels)} {len(labels)}\n{labels}")
# '390': False, '135': False, '136': True, '137': True, '138': True, '139': False}